In [4]:
import tensorflow as tf
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing import sequence
from tensorflow.python.keras.layers import *
from keras.models import Sequential

Using TensorFlow backend.


In [0]:

# Number of words to consider as features
max_features = 10000
# Cut texts after this number of words (among top max_features most common words)
maxlen = 500

# Load data
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)

# Reverse sequences
x_train = [x[::-1] for x in x_train]
x_test = [x[::-1] for x in x_test]

# Pad sequences
x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)

In [0]:
def make_model():
  model = tf.keras.Sequential()
  model.add(Embedding(10000,64))
  model.add(SimpleRNN(32))
  model.add(Dense(1,activation='sigmoid'))
  model.compile(
      optimizer='adam',
      loss='binary_crossentropy',
      metrics=['acc'])
  return model
   

In [0]:
import os
# This address identifies the TPU we'll use when configuring TensorFlow.
TPU_WORKER = 'grpc://' + os.environ['COLAB_TPU_ADDR']


In [10]:
# tf.config.experimental_connect_to_host('grpc://' + os.environ['COLAB_TPU_ADDR'])
# resolver = tf.distribute.cluster_resolver.TPUClusterResolver('grpc://' + os.environ['COLAB_TPU_ADDR'])
# tf.tpu.experimental.initialize_tpu_system(resolver)
# strategy = tf.distribute.experimental.TPUStrategy(resolver) 


tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
print('Running on TPU ', tpu.cluster_spec().as_dict()['worker'])

tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)
strategy = tf.distribute.experimental.TPUStrategy(tpu)

print("REPLICAS: ", strategy.num_replicas_in_sync)

Running on TPU  ['10.103.100.130:8470']


INFO:tensorflow:Initializing the TPU system: grpc://10.103.100.130:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.103.100.130:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


REPLICAS:  8


In [16]:
with strategy.scope():
  training_model = make_model()
  training_model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, None, 64)          640000    
_________________________________________________________________
simple_rnn_1 (SimpleRNN)     (None, 32)                3104      
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 33        
Total params: 643,137
Trainable params: 643,137
Non-trainable params: 0
_________________________________________________________________


In [17]:
import time
start_time = time.time()

history = training_model.fit(x_train, y_train,
                    epochs=20,
                    batch_size=128 * 8,
                    validation_split=0.2)
training_model.save_weights('tpu_model.h5', overwrite=True)

print("--- %s seconds ---" % (time.time() - start_time))


Epoch 1/20
20/20 [==============================] - 5s 242ms/step - acc: 0.5466 - loss: 0.6830 - val_acc: 0.6448 - val_loss: 0.6396
Epoch 2/20
20/20 [==============================] - 4s 184ms/step - acc: 0.7321 - loss: 0.5569 - val_acc: 0.7832 - val_loss: 0.5019
Epoch 3/20
20/20 [==============================] - 4s 183ms/step - acc: 0.8268 - loss: 0.4341 - val_acc: 0.8072 - val_loss: 0.4562
Epoch 4/20
20/20 [==============================] - 4s 195ms/step - acc: 0.8735 - loss: 0.3342 - val_acc: 0.8148 - val_loss: 0.4308
Epoch 5/20
20/20 [==============================] - 4s 181ms/step - acc: 0.8963 - loss: 0.2807 - val_acc: 0.8260 - val_loss: 0.4247
Epoch 6/20
20/20 [==============================] - 4s 185ms/step - acc: 0.9154 - loss: 0.2396 - val_acc: 0.8352 - val_loss: 0.4084
Epoch 7/20
20/20 [==============================] - 4s 192ms/step - acc: 0.9483 - loss: 0.1689 - val_acc: 0.8236 - val_loss: 0.4314
Epoch 8/20
20/20 [==============================] - 4s 186ms/step - acc: 0.9

In [21]:
training_model.load_weights("tpu_model.h5")
print(training_model.evaluate(x_test,y_test))

782/782 [==============================] - 10s 12ms/step - acc: 0.7832 - loss: 0.6828
[0.6827979683876038, 0.7831999659538269]
